# Kotlin NDArrays

Thanks to **Multik** and the standard Collections library, it's easy to create, access and manipulate N-dimensional arrays and matrices.

You can import Multik using Jupyter's magic command `%use`, for importing Multik library and dependencies.

In [1]:
q%use multik

We can create a simple 1D array as follows (type declaration is not mandatory)

In [2]:
val arr: NDArray<Int, D1> = mk.ndarray(mk[1, 2, 3])
arr

[1, 2, 3]

Notice the type definition of the Array, where we must specify:
- **Type**: Any Kotlin subtype of `Number` (`Boolean` is not permitted).
- **Dimension**: Multik defines 5 `Dimension` types: `D1`, `D2`, `D3`, `D4`, `DN`. `DN` is used in all the cases where the dimension is not known in advance, or when the dimension size is major than 4.

In the same way, we can create a 2 dimensional array, but this time we let the compiler infer the type and dimension

In [3]:
val arr = mk.ndarray(mk[mk[1, 2, 3], mk[4, 5, 6]])
arr

[[1, 2, 3],
[4, 5, 6]]

We can also create an array from *Kotlin Collections* **List** or **Set**

In [4]:
val arr = mk.ndarray(listOf(1, 2, 3))
arr

[1, 2, 3]

## Creation

We will go through simple `NDArrays` creation methods.

Creating equally spaced arrays with `arange` and `linspace`

In [5]:
val a = mk.linspace<Double>(0, 1, 5)
val b = mk.arange<Int>(0, 10, 2)

println("a -> $a")
println("b -> $b")

a -> [0.0, 0.25, 0.5, 0.75, 1.0]
b -> [0, 2, 4, 6, 8]


Generating an array of Zeros and Ones

In [6]:
val zrs = mk.zeros<Double>(10)
val ons = mk.ones<Double>(10)
println("Zeros ->$zrs")
println("Ones -> $ons")

Zeros ->[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Ones -> [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


We can then map the array to a matrix using the `reshape` method, which takes in input an arbitrary number of dimensions size along which the array will be mapped. 

In [7]:
mk.zeros<Int>(50).reshape(2, 5, 5) // three dimensional matrix with (z, x, y) = (2, 5, 5)

[[[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0]],

[[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0]]]

It's also possible to create an `NDArray` providing a **lambda** for constructing it, considering that the matrix will be built upon a `arange` vector.

In [8]:
mk.d3array(2, 5, 5) { it % 2 }

[[[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0]],

[[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1]]]

Same as

In [9]:
mk.arange<Int>(50).map { it % 2 }.reshape(2, 5, 5)

[[[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0]],

[[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1]]]

### Arithmetic with NDArrays

In the current version of Multik, an `NDArray` object support all arithmetic operations, enabling what are called as **vectorized** operations.
Vectorization is very convenient because enable the user to express batch operations on data without writing any for loops (and also could possibly enhance performances).

In [12]:
val arr = mk.ndarray(mk[mk[1.0, 2.0, 3.0], mk[4.0, 5.0, 6.0]])
arr

[[1.0, 2.0, 3.0],
[4.0, 5.0, 6.0]]

In [36]:
arr * arr

[[1.0, 4.0, 9.0],
[16.0, 25.0, 36.0]]

In [37]:
1.0 / arr

[[1.0, 0.5, 0.3333333333333333],
[0.25, 0.2, 0.16666666666666666]]

In [76]:
arr dot arr.transpose()

[[14.0, 32.0],
[32.0, 77.0]]

````{margin}
```{warning}
Operations among equal shaped arrays and matrices are always allowed. As the current version of Multik (v0.2.0), there is no native support for array **broadcasting**.
In section K (some ref to where i could explain how to) I explain how to archieve array broadcasting using tensor algebra provided by [Kmath](https://github.com/SciProgCentre/kmath)
```
````

Multik supports two separate packages for linear algebra basic operations (package `api.linalg`) and vectorized math operations (package `api.math`)

In [234]:
mk.math.cumSum(arr, axis = 1)

[[1.0, 3.0, 6.0],
[4.0, 9.0, 15.0]]

In [235]:
val sqrMat = mk.ones<Double>(3, 3)
// Frobenius norm of the matrix (default when calling `norm()`)
mk.linalg.norm(sqrMat, norm = Norm.Fro)

3.0

In [236]:
mk.math.sin(arr)

[[0.8414709848078965, 0.9092974268256817, 0.1411200080598672],
[-0.7568024953079283, -0.9589242746631385, -0.27941549819892586]]

The `linalg` package contains also methods for solving linear matrix equations and matrix decomposition methods.

In [237]:
// solve the linear system
val a = mk.d2array(3, 3) { it * it }.asType<Double>()
val b = mk.ndarray(mk[54.0, 396.0, 1062.0])

val res = mk.linalg.solve(a, b).map { it.roundToInt() }
println(" x = $res")

// check
(a dot res.asType<Double>()) == b


 x = [0, 6, 12]


true

For more, visit Multik's documentation for [`linalg`](https://kotlin.github.io/multik/multik-core/org.jetbrains.kotlinx.multik.api.linalg/index.html) and [`math`](https://kotlin.github.io/multik/multik-core/org.jetbrains.kotlinx.multik.api.math/index.html)

### Indexing and Slicing

There are many ways to select subset of data of and `NDArray`.

For one dimensional arrays is straightforward:

In [240]:
val arr = mk.arange<Int>(10)
arr

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [241]:
// basic indexing
arr[2]

2

In [268]:
// range indexing
println(arr[1..3])
// the behavior is different using kotlin keyword `until`
println(arr[1 until 3])

[1, 2, 3]
[1, 2]


For Multi-Index arrays the *range* operator can be used like:

In [297]:
val a = mk.linspace<Int>(0, 20, 10).reshape(5, 2)

// single elemt selection
a[3, 1]
// row selection
a[0]
// range selection left inclusive
a[0..2] // != a[0 until 2] -> left exlusive
// selecting first column of first 3 rows
a[0..2, 0]

[0, 4, 8]

```{warning}
Slices are **copies** of the source array, unlike NumPy that generates a *view* on the original array.
```kotlin
var b = a[0..2, 0]
a[0..2, 0] === b // false!
```
```

`NDArrays` does not support boolean indexing, but filters can be applied with the `filter()` method, like every `Collection` in Kotlin's standard library.

For each "indexed functional method" that can be applied to kotlin collections, like `mapIndexed`, `forEachIndexed`, `filterIndexed`, Multik offers the counterpart for multidimensional arrays (of the form `*MultiIndexed()`), where the index is an `intArray()` of the combination of the indices of the current element.

A full list of those methods, and more, can be found int the [`ndarray.operations`](https://kotlin.github.io/multik/multik-core/org.jetbrains.kotlinx.multik.ndarray.operations/index.html) package.